# Flight Planning Domain 

This notebook aims to make a short and interactive example of the Flight Planning Domain. See the [online documentation](https://airbus.github.io/scikit-decide/reference/_skdecide.hub.domain.flight_planning.domain.html#flightplanningdomain) for more information.

Concerning the python kernel to use for this notebook:
- If running locally, be sure to use an environment with scikit-decide[all].
- If running on colab, the next cell does it for you.
- If running on binder, the environment should be ready.

In [ ]:
# # On Colab: install the library
# on_colab = "google.colab" in str(get_ipython())
# if on_colab:
#     import glob
#     import json
#     import sys

#     using_nightly_version = True

#     if using_nightly_version:
#         # look for nightly build download url
#         release_curl_res = !curl -L   -H "Accept: application/vnd.github+json" -H "X-GitHub-Api-Version: 2022-11-28" https://api.github.com/repos/airbus/scikit-decide/releases/tags/nightly
#         release_dict = json.loads(release_curl_res.s)
#         release_download_url = sorted(
#             release_dict["assets"], key=lambda d: d["updated_at"]
#         )[-1]["browser_download_url"]
#         print(release_download_url)

#         # download and unzip
#         !wget --output-document=release.zip {release_download_url}
#         !unzip -o release.zip

#         # get proper wheel name according to python version used
#         wheel_pythonversion_tag = f"cp{sys.version_info.major}{sys.version_info.minor}"
#         wheel_path = glob.glob(
#             f"dist/scikit_decide*{wheel_pythonversion_tag}*manylinux*.whl"
#         )[0]

#         skdecide_pip_spec = f"{wheel_path}[all]"
#     else:
#         skdecide_pip_spec = "scikit-decide[all]"

#     # uninstall google protobuf conflicting with ray and sb3
#     ! pip uninstall -y protobuf

#     # install scikit-decide with all extras
#     !pip install {skdecide_pip_spec}

In [ ]:
import datetime

from pygeodesy.ellipsoidalVincenty import LatLon

from skdecide.hub.domain.flight_planning.aircraft_performance.bean.aircraft_state import (
    AircraftState,
)
from skdecide.hub.domain.flight_planning.aircraft_performance.performance.performance_model_enum import (
    PerformanceModelEnum,
)
from skdecide.hub.domain.flight_planning.aircraft_performance.performance.phase_enum import (
    PhaseEnum,
)
from skdecide.hub.domain.flight_planning.aircraft_performance.performance.rating_enum import (
    RatingEnum,
)
from skdecide.hub.domain.flight_planning.domain import FlightPlanningDomain, WeatherDate
from skdecide.hub.domain.flight_planning.flightplanning_utils import plot_graph_vertical, plot_network_adapted, plot_flight_graph_on_map_with_edges
from skdecide.hub.solver.astar import Astar

from openap.extra.nav import airport
from openap.extra.aero import cas2mach, ft, kts
import numpy as np


# reload
%load_ext autoreload
%autoreload 2

## Definition of the problem

Here we will make a short haul flight from Paris Charles de Gaulle airport (ICAO : LFPG) to Toulouse-Blagnac airport (ICAO: LFBO), using an airbus A320 aircraft.

In [ ]:
origin = "LFPG"
destination = "LFBO"
aircraft = "A320"

Now, we are going to define a date that will be used for weather interpolation. If the data has not already be downloaded on your computer, be careful to choose a date within the past 6 months, so that the data is available on https://www.ncei.noaa.gov.

In [ ]:
# we set a date valid for 4 months to avoid downloading weather data at each daily run.
today = datetime.date.today()
month = ((today.month) - 1) // 4 * 4 + 1  # will result in january, may, or september
year = today.year
day = 1

weather_date = WeatherDate(day=day, month=month, year=year)
print(weather_date)

We can now define heuristic and cost function, to feed the A* solver. This aims to guide him along the airways graph to solve the problem, and get as close as possible to the optimal solution. 

The heuristic function can be either `"fuel"`, `"time"`, `"distance"`, `"lazy_fuel"`, `"lazy_time"`, or `None`. If `None`, the A* will behave as a Dijkstra-like search algorithm, as we give a 0 value to the A* algorithm. 

The cost function can be either `"fuel"`, `"time"`, or `"distance"`. 
It will define the cost of the flight plan, computed during the state-to-state flight simulation. 

In [ ]:
heuristic = "fuel"
cost_function = "fuel"

## Definition of the aircraft initial state
Now, we define the aircraft state. This implies picking an aircraft performance model : `OPENAP` or `POLL_SCHUMANN`

In [ ]:
acState = AircraftState(
    model_type="A320",  # only for OPENAP and POLL_SCHUMANN
    performance_model_type=PerformanceModelEnum.POLL_SCHUMANN,  # PerformanceModelEnum.OPENAP
    gw_kg=80_000,
    zp_ft=10_000,
    mach=cas2mach(250*kts, h=10_000*ft, dT=0),
    phase=PhaseEnum.CLIMB,
    rating_level=RatingEnum.MCL,
    cg=0.3,
)

## Definition of the corresponding domain

We are now going to define the domain. It can take some time, especially if it needs to download some weather files, or if you ask for a fuel loop. 


In [ ]:
domain_factory = lambda: FlightPlanningDomain(
    aircraft_state=acState,

    mach_cruise=0.78,
    mach_climb=0.7,
    mach_descent=0.65,

    nb_forward_points=20,
    nb_lateral_points=10,
    nb_climb_descent_steps=5,
    flight_levels_ft=list(np.arange(30_000, 38_000+2_000, 2_000)),
    graph_width="medium",
    origin=LatLon(43.629444, 1.363056),
    destination="EDDB",

    objective="fuel",
    heuristic_name="lazy_fuel",

    weather_date=weather_date
)

domain = domain_factory()

We can plot the graph:

In [ ]:
plot_network_adapted(
    graph=domain.network,
    p0=LatLon(43.629444, 1.363056),
    p1=LatLon(airport("EDDB")["lat"], airport("EDDB")["lon"], airport("EDDB")["alt"]*ft)
)

## Solving and rendering out the flight planning domain

We use here an A* solver as mentionned before. 
We also use the custom rollout proposed to have some visualization during the flight planning generation.

In [ ]:
solver = Astar(
    domain_factory=domain_factory,
    heuristic=lambda d, s: d.heuristic(s),
    parallel=False,
)

In [ ]:
solver.solve()

In [ ]:
domain.custom_rollout(solver=solver, make_img=True)

In [ ]:
domain.observation.trajectory

## Aircraft state initial location somewhere on the graph

In [ ]:
acState = AircraftState(
    model_type="A320",  # only for OPENAP and POLL_SCHUMANN
    performance_model_type=PerformanceModelEnum.POLL_SCHUMANN,  # PerformanceModelEnum.OPENAP
    gw_kg=80_000,
    zp_ft=18000.0,
    mach=cas2mach(250*kts, h=10_000*ft, dT=0),
    phase=PhaseEnum.CLIMB,
    rating_level=RatingEnum.MCL,
    cg=0.3,
    x_graph=5,
    y_graph=5,
    z_graph=10
)

In [ ]:
domain_factory = lambda: FlightPlanningDomain(
    aircraft_state=acState,

    mach_cruise=0.78,
    mach_climb=0.7,
    mach_descent=0.65,

    nb_forward_points=20,
    nb_lateral_points=10,
    nb_climb_descent_steps=5,
    flight_levels_ft=list(np.arange(30_000, 38_000+2_000, 2_000)),
    graph_width="medium",
    origin=LatLon(43.629444, 1.363056),
    destination="EPWA",

    objective="fuel",
    heuristic_name="lazy_fuel",

    weather_date=weather_date
)

domain = domain_factory()

In [ ]:
solver = Astar(
    domain_factory=domain_factory,
    heuristic=lambda d, s: d.heuristic(s),
    parallel=False,
)

In [ ]:
solver.solve()

In [ ]:
domain.custom_rollout(solver=solver, make_img=True)

In [ ]:
domain.observation.trajectory

In [ ]:
plot_flight_graph_on_map_with_edges(
    G=domain.network,
    nb_lateral_points_param=10)